In [ ]:
import csv
import sys
import re
import matplotlib.pyplot as plt
import numpy as np

# Cutoff for rare variants 

In [ ]:
cutoff  = 0.0000001

In [112]:
def writelist (mylist, myfile): 
    fp =  open(myfile,'a+')
    for idx,item in enumerate(mylist):
            fp.write(item)
            fp.write('\t')
    fp.write('\n')
    fp.close()

In [ ]:
def intersections(list1, list2):
    intersection_list = []
    for i in list1: 
        if i in list2: 
                intersection_list.append(i)
    return intersection_list

In [ ]:
def difference(list1, list2):
    unique_list = []
    for i in list1: 
        if i not in list2: 
                unique_list.append(i)
    return unique_list

In [ ]:
def exist (list1, list2): 
    found_list = []
    for i in list1: 
            if i not in found_list: 
                    found_list.append(i)
    for i in list2: 
            if i not in found_list: 
                    found_list.append(i)
    return found_list 

In [115]:
def common_variants(variants1, variants2, outfile): 
        fp = open(outfile, 'w')
        for i in variants1: 
               for j in variants2: 
                     count = 0 
                     for a in range(0,12): 
                            if i[a] == j[a]:
                                count +=1
                     if count == 12: 
                          print>>fp, i 

# Read annovar annotation file

In [ ]:
 def load_annovar(filename):
    rows = []
    exacall = []
    gnomall = []
    rs = []
    gene = []
    pathogenics = []
    functions = []
    exonicfunctions =[]
    fp = open(filename, 'r')
    r = csv.reader(fp, delimiter='\t')
    for n, row in enumerate(r):
        if n == 0: 
                continue
        exonicfunctions.append(row[8])
        exacall.append(row[11])
        gnomall.append(row[119])
        rs.append(row[19])
        rows.append(row)
        gene.append(row[6])
        functions.append(row[5])
        pathogenics.append(row[54])
    return rows, functions, exonicfunctions, rs, gene,  exacall , gnomall, pathogenics

# Filter for rare MAF

In [83]:
def maf_filter(mfile): 
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    outfile = mfile.split('.')[0] + '_raremaf.txt'
    raremaf_genes = []
    variants = []
    variants_list = [] 
    for i in range(0,len(exacall) ): 
            #========================================
            #If both gnomAD and ExAC is '.'
            #========================================
            if (gnomall[i] != '.') and (exacall[i] !='.'):
                if   (float(gnomall[i]) < cutoff) and (float(exacall[i]) < cutoff) and functions[i] =='exonic': 
                    count += 1 
                    writelist(rows[i], outfile)
                    variants_list.append(rows[i])
                    print(rs[i], gene[i], exacall[i],gnomall[i])
                    if gene[i] not in raremaf_genes: 
                            raremaf_genes.append(gene[i])
                    var = rs[i]+'.'+gene[i]
                    if var not in variants: 
                            variants.append(var)
            #========================================
            #If both gnomAD is '.' and ExAC is not '.'
            #========================================               
            elif (gnomall[i] == '.') and (exacall[i] != '.') and (float(exacall[i]) < cutoff) and functions[i] =='exonic':
                    count += 1 
                    writelist(rows[i], outfile)                    
                    variants_list.append(rows[i])
                    print(rs[i], gene[i], exacall[i],gnomall[i])
                    if gene[i] not in raremaf_genes: 
                            raremaf_genes.append(gene[i])
                    var = rs[i]+'.'+gene[i]
                    if var not in variants: 
                            variants.append(var)
            #========================================
            #If both ExAC is '.' and gnomAD is not '.'
            #======================================== 
            elif (exacall[i] =='.') and (gnomall[i] != '.') and (float (gnomall[i]) < cutoff) and functions[i] =='exonic':
                    count += 1 
                    writelist(rows[i], outfile)
                    variants_list.append(rows[i])
                    print(rs[i], gene[i], exacall[i],gnomall[i])
                    if gene[i] not in  raremaf_genes: 
                             raremaf_genes.append(gene[i])
                    var = rs[i]+'.'+gene[i]
                    if var not in variants: 
                            variants.append(var)
    return(raremaf_genes,  variants_list)


# Filter for pathogenic variants

In [ ]:
def pathogenic_filter(mfile):
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    pathogenic_genes = []
    outfile = mfile.split('.')[0]+'pathogenic.txt'
    for i in range(0,len(exacall) ): 
        if 'athogenic' in pathogenics[i]: 
            if (exacall[i] == '.') or( float(exacall[i]) < cutoff) or (gnomall[i] =='.') or (float(gnomall[i]) < cutoff):
                count += 1 
                writelist(rows[i], outfile)
                print(rs[i], gene[i], exacall[i],gnomall[i], pathogenics[i])
                if gene[i] not in pathogenic_genes:
                        pathogenic_genes.append(gene[i])
    print ('Number of variants that are rare and pathogenic cutoff <', cutoff, 'is:',count)
    return pathogenic_genes 

# Filter for no MAF variants

In [78]:
def nomaf_filter(mfile):    
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    nomafs_genes = []
    nomafvariants = [] 
    outfile = mfile.split('.')[0]+'nomaf.txt'
    
    for i in range(0,len(exacall) ): 
        if (exacall[i] == '.') and (gnomall[i] =='.') and functions[i] =='exonic':
                count += 1 
                writelist(rows[i], outfile)
                if gene[i] not in nomafs_genes:
                        nomafs_genes.append(gene[i])
                var = rs[i]+'.'+gene[i]
                if var not in  nomafvariants: 
                         nomafvariants.append(var)
    print ('Number of variants with no MAf in both exacall and gnomAD <', cutoff, 'is:',count)
    return nomafs_genes,  nomafvariants 

# F366 Analysis

In [84]:
f366m1_raremaf, f366m1_variants  = maf_filter('f366m1-anno-ref.hg19_multianno.txt')

('rs543315719', 'CRYBG2', '0', '.')
('rs146527074', 'GSTM1', '0', '0')
('rs79475403', 'IGSF3', '.', '0')
('rs200427142', 'IGFN1', '.', '0')
('.', 'ITPKB', '.', '0')
('rs1057389', 'CGREF1', '.', '0')
('rs376333361', 'TEKT4', '.', '0')
('rs142133592', 'TEKT4', '.', '0')
('rs78179792', 'ANKRD36C', '.', '0')
('rs62152963', 'ADRA2B', '0', '.')
('.', 'ANKRD36B', '.', '0')
('rs71624922', 'SYN2', '0', '.')
('rs368284573', 'PLCL2', '0', '.')
('.', 'GPX1', '.', '0')
('rs199810415', 'FRG2C', '0', '.')
('rs62247960', 'FRG2C', '0', '.')
('rs148249548', 'ZNF717', '.', '0')
('rs76921234', 'ZNF717', '0', '.')
('rs149887072', 'ZNF717', '.', '0')
('rs112847134', 'ZNF717', '.', '0')
('rs74861398', 'ZNF717', '.', '0')
('rs2918520', 'ZNF717', '.', '0')
('rs370790767', 'ADRA2C', '.', '0')
('rs756096374', 'KIAA1211', '0', '0')
('.', 'SOWAHA', '.', '0')
('.', 'PCDHA8', '.', '0')
('rs28421695', 'WWC1', '.', '0')
('rs199837247', 'FAM8A1', '.', '0')
('rs758886973', 'FAM8A1', '0', '0')
('.', 'HLA-DRB5', '.', '0')

In [85]:
f366m2_raremaf , f366m2_variants= maf_filter('f366m2-anno-ref.hg19_multianno.txt')

('rs79475403', 'IGSF3', '.', '0')
('rs200427142', 'IGFN1', '.', '0')
('rs376333361', 'TEKT4', '.', '0')
('rs142133592', 'TEKT4', '.', '0')
('rs78179792', 'ANKRD36C', '.', '0')
('rs62152963', 'ADRA2B', '0', '.')
('rs372700939', 'IGFBP2', '.', '0')
('rs377689849', 'IGFBP2', '.', '0')
('rs185489391', 'IGFBP2', '.', '0')
('rs71624922', 'SYN2', '0', '.')
('rs368284573', 'PLCL2', '0', '.')
('rs199810415', 'FRG2C', '0', '.')
('rs62247960', 'FRG2C', '0', '.')
('rs148249548', 'ZNF717', '.', '0')
('rs76921234', 'ZNF717', '0', '.')
('rs149887072', 'ZNF717', '.', '0')
('rs112847134', 'ZNF717', '.', '0')
('rs74861398', 'ZNF717', '.', '0')
('rs2918520', 'ZNF717', '.', '0')
('.', 'OR5H14', '.', '0')
('rs370790767', 'ADRA2C', '.', '0')
('rs756096374', 'KIAA1211', '0', '0')
('rs778356568', 'FRG1', '0', '0')
('.', 'GRK6', '.', '0')
('.', 'OR12D2', '.', '0')
('.', 'HLA-DRB1', '.', '0')
('.', 'HLA-DRB1', '.', '0')
('.', 'HLA-DRB1', '.', '0')
('rs34133353', 'FOXO3', '.', '0')
('rs75141335', 'MUC3A', '0', '

In [ ]:
f366m1_pathogenic  = pathogenic_filter('f366m1-anno-ref.hg19_multianno.txt')

In [ ]:
f366m2_pathogenic = pathogenic_filter('f366m2-anno-ref.hg19_multianno.txt')

In [ ]:
f366m1_nomaf = nomaf_filter('f366m1-anno-ref.hg19_multianno.txt')

In [ ]:
f366m2_nomaf = nomaf_filter('f366m2-anno-ref.hg19_multianno.txt')

In [117]:
common_variants(f366m1_variants,f366m2_variants, 'f366m1m2.rare.common.txt')

In [106]:
f366m1_m2_raremaf = intersections(f366m2_raremaf,f366m1_raremaf)
print(f366m1_m2_raremaf)
write_variants(f366m1_m2_raremaf, 'f366m1-anno-ref.hg19_multianno.txt', 'f366m1-anno-ref.hg19_multianno.txt', 'f366.rare.common.txt') 

['IGSF3', 'IGFN1', 'TEKT4', 'ANKRD36C', 'ADRA2B', 'SYN2', 'PLCL2', 'FRG2C', 'ZNF717', 'ADRA2C', 'KIAA1211', 'FOXO3', 'MUC3A', 'MUC17', 'KBTBD11', 'PCMTD1', 'MEGF9', 'CTBP2', 'MUC6', 'MUC5B', 'MTCH2', 'MAML2', 'VWF', 'CLECL1', 'GXYLT1', 'FAM186A', 'KRT1', 'TDG', 'MMP17', 'PABPC3', 'ERICH6B', 'GOLGA6L2', 'BNC1', 'APOBR', 'SULT1A2', 'HYDIN', 'KRTAP9-1', 'CDC27', 'CABLES1', 'MUC16', 'ZNF681', 'NUMBL', 'ZNF534', 'KIR3DL3', 'KIR2DL4', 'KIR3DL1', 'KIR2DS4', 'PLAC4', 'FAM47A', 'SLC25A5']


# F366 pathogenic variants common between M1 and M2

In [ ]:
f366m1_m2_pathogenic = intersections(f366m1_pathogenic , f366m2_pathogenic )
print(f366m1_m2_pathogenic )

# F366  no MAFs common between M1 and M2

In [ ]:
f366m1_m2_nomaf   = difference(f366m1_nomaf  , f366m2_nomaf  )
print(f366m1_m2_nomaf )